<a href="https://colab.research.google.com/github/olgasem10/Compling-tasks/blob/master/JIGSAW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Экзамен по машинному обучению

Пункты из всех рядов кроме сабмитов

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.set_option('display.max_colwidth', 100)

In [ ]:
data = pd.read_csv('jigsaw-toxic-comment-train.csv')

In [ ]:
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They wer...,0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, Januar...",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relev...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0


# Описательный анализ

Ряд 1, колонка 2: Описательный анализ данных с привязкой к целевой переменной (toxic) (как минимум 5 статистик на каждую группу) - 1.25 балл

In [ ]:
stats = data

 Длина комментария в символах

In [ ]:
stats['text_char_length'] = stats['comment_text'].apply(lambda x: len(x))

Длина комментария в токенах

In [ ]:
stats['text_word_length'] = stats['comment_text'].apply(lambda x: len(x.split()))

Длина комментария в абзацах

In [ ]:
stats['text_par_length'] = stats['comment_text'].apply(lambda x: len(x.split('\n')))

Средняя длина слова в комментарии

In [ ]:
stats['mean_word_length'] = stats['comment_text'].apply(lambda x: np.mean([len(word) for word in x.split()]))

Доля заглавных букв

In [ ]:
stats['upper_ratio'] = stats['comment_text'].apply(lambda x: len([ch for ch in x if ch.isupper()])/len(x))

Доля небуквенных символов

In [ ]:
stats['nonalpha_ratio'] = stats['comment_text'].apply(lambda x: len([ch for ch in x if not ch.isalpha() and not ch == ' '])/len(x))

Сравнение токсичных и не токсичных комментариев:

In [ ]:
stats[(stats['toxic'] == 1)][['text_char_length', 'text_word_length', 
                              'text_par_length', 'mean_word_length', 
                              'upper_ratio', 'nonalpha_ratio']].describe()

,text_char_length,text_word_length,text_par_length,mean_word_length,upper_ratio,nonalpha_ratio
count,21384.000000,21384.000000,21384.000000,21384.000000,21384.000000,21384.000000
mean,280.604097,48.573466,3.213384,5.121478,0.112577,0.073643
std,613.588485,105.424224,8.807426,20.165753,0.211337,0.077752
min,3.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,55.000000,10.000000,1.000000,4.041667,0.017964,0.031250
50%,115.000000,21.000000,1.000000,4.461538,0.034206,0.052632
75%,250.000000,45.000000,3.000000,4.928571,0.068182,0.089552
max,5000.000000,2321.000000,289.000000,1655.000000,0.998189,0.994769


In [ ]:
stats[(stats['toxic'] == 0)][['text_char_length', 'text_word_length', 
                              'text_par_length', 'mean_word_length', 
                              'upper_ratio', 'nonalpha_ratio']].describe()

,text_char_length,text_word_length,text_par_length,mean_word_length,upper_ratio,nonalpha_ratio
count,202165.000000,202165.000000,202165.000000,202165.000000,202165.000000,202165.000000
mean,402.691178,68.415161,3.599322,4.959716,0.045593,0.070867
std,589.421496,98.633811,6.362831,4.414257,0.070363,0.066595
min,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,100.000000,17.000000,1.000000,4.350000,0.020408,0.034483
50%,215.000000,37.000000,2.000000,4.714286,0.031250,0.052239
75%,450.000000,77.000000,3.000000,5.125000,0.047619,0.082645
max,5000.000000,1990.000000,596.000000,708.714286,1.000000,1.000000


Можно отметить, что токсичные комментарии в среднем короче, чем нетоксичные (и по символам, и по токенам, и по абзацам), но при этом у токсичных во всех случаях больше отклонение

Самый длинный в токенах комментарий - токсичный, самый длинный в абзацах - нетоксичный

Средняя длинна токена у токсичных комментариев больше

В токсичных комментариях гораздо выше доля содержания заглавных букв

Что касается содержания небуквенных символов (кроме пробелов) то их доля мало отличается для токсичных и нетоксичных комментариев

# Бейзлайн модель с отбором признаков

Ряд 2, колонка 5: Бейзлайн модель из sklearn (векторайзер + модель) с отбором признаков (через permutation importance) - 2 балл

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
X = data.comment_text.values
y = data.toxic.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state=42, stratify = y)

In [ ]:
tfidf = TfidfVectorizer(max_features=10000, min_df=5, ngram_range=(1,2), max_df=0.4)

In [ ]:
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

In [ ]:
logreg = LogisticRegression(max_iter = 200).fit(X_train_tfidf, y_train)

In [ ]:
predicted = logreg.predict(X_test_tfidf)
probabilities = logreg.predict_proba(X_test_tfidf)
probs = [p[1] for p in probabilities]

In [ ]:
def metrics(true, pred, probs):
    acc = accuracy_score(true, pred)
    p = precision_score(true, pred, average='binary')
    r = recall_score(true, pred, average='binary')
    f1 = f1_score(true, pred, average='binary')
    roc_auc = roc_auc_score(true, probs)
    print('acc = {0:1.4f}'.format(acc))
    print('F1 = {0:1.4f}, P = {1:1.4f}, R = {2:1.4f}'.format(f1, p, r))
    print('ROC AUC = {0:1.4f}'.format(roc_auc))

In [ ]:
metrics(y_test, predicted, probs)

acc = 0.9530
F1 = 0.7096, P = 0.8681, R = 0.6001
ROC AUC = 0.9664


### Отбор признаков

In [ ]:
%%time
perm = PermutationImportance(logreg, random_state=1).fit(X_train_tfidf.toarray(), y_train)

CPU times: user 11h 28min 38s, sys: 1h 22min 29s, total: 12h 51min 7s
Wall time: 7h 49min 11s


In [ ]:
eli5.show_weights(perm, top=10)

Weight,Feature
0.0083 ± 0.0006,x3285
0.0049 ± 0.0006,x3300
0.0039 ± 0.0002,x7152
0.0032 ± 0.0003,x7540
0.0022 ± 0.0002,x3866
0.0019 ± 0.0002,x1402
0.0019 ± 0.0001,x7576
0.0016 ± 0.0001,x1042
0.0014 ± 0.0002,x1052
0.0011 ± 0.0001,x6106


In [ ]:
weights = eli5.formatters.as_dataframe.explain_weights_df(perm)
weights.to_csv('weights10000.csv', index = False)

In [ ]:
top_features = [int(i[1:]) for i in weights[weights.weight != 0].feature if 'BIAS' not in i]

Из 10000 признаков после отбора осталось 4430

In [ ]:
len(top_features)

4430

In [ ]:
weights[weights.weight == 0]

,feature,weight,std
2647,x3524,0.0,0.0
2648,x3814,0.0,0.0
2649,x3476,0.0,0.0
2650,x585,0.0,0.0
2651,x747,0.0,0.0
...,...,...,...
8212,x5112,0.0,0.0
8213,x6429,0.0,0.0
8214,x5044,0.0,0.0
8215,x5116,0.0,0.0


In [ ]:
X_train_perm = X_train_tfidf[:,top_features]
X_test_perm = X_test_tfidf[:,top_features]

In [ ]:
perm_model = LogisticRegression(max_iter = 200).fit(X_train_perm, y_train)

In [ ]:
predicted2 = perm_model.predict(X_test_perm)
probabilities2 = perm_model.predict_proba(X_test_perm)
probs2 = [p[1] for p in probabilities2]

In [ ]:
metrics(y_test, predicted2, probs2)

acc = 0.9531
F1 = 0.7105, P = 0.8677, R = 0.6015
ROC AUC = 0.9663


# Ансамбль

Ряд 3, колонка 5: Ансамбль из моделей в sklearn - 2 балла

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import FunctionTransformer

In [ ]:
clf1 = LogisticRegression(max_iter = 200)
clf2 = GaussianNB()
clf3 = LinearSVC(max_iter = 500)
clf4 = DecisionTreeClassifier()
clf5 = RidgeClassifier()

In [ ]:
voting = VotingClassifier(estimators=[
        ('clf1', clf1), 
        ('clf2', clf2), 
        ('clf3', clf3), 
        ('clf4', clf4),
        ('clf5', clf5) 
        ], 
        voting='hard')

In [ ]:
%%time
pipe = Pipeline([
    ('to_dense', FunctionTransformer(lambda x: x.todense(), accept_sparse=True)), 
    ('clf', voting),
    ])
pipe = pipe.fit(X_train_perm[:80000], y_train[:80000])

CPU times: user 42min 19s, sys: 7.16 s, total: 42min 26s
Wall time: 40min 45s


In [ ]:
def metrics(true, pred):
    acc = accuracy_score(true, pred)
    p = precision_score(true, pred, average='binary')
    r = recall_score(true, pred, average='binary')
    f1 = f1_score(true, pred, average='binary')
    roc_auc = roc_auc_score(true, pred)
    print('acc = {0:1.4f}'.format(acc))
    print('F1 = {0:1.4f}, P = {1:1.4f}, R = {2:1.4f}'.format(f1, p, r))
    print('ROC AUC = {0:1.4f}'.format(roc_auc))

In [ ]:
predictions2 = pipe.predict(X_test_perm)

In [ ]:
metrics(y_test, predictions2)

acc = 0.9533
F1 = 0.7102, P = 0.8747, R = 0.5978
ROC AUC = 0.7944


Рассчитать ROC AUC по-настоящему здесь не получается, поскольку при параметре voting = 'hard' вероятности недоступны, а при 'soft' голосовании ансамбль работал хуже

# Токенизация на символьные n-граммы

Ряд 5, колонка 5: Использование токенизации на символьные нграммы - 2 балла

In [ ]:
import youtokentome as yttm

In [ ]:
yttm_train = '\n'.join(X_train.tolist())

In [ ]:
train_path = '/content/drive/My Drive/Jigsaw/yttm_train2.txt'
model_path = '/content/drive/My Drive/Jigsaw/yttm2.model'

In [ ]:
with open(train_path, 'w') as f:
  f.write(yttm_train)

In [ ]:
yttm.BPE.train(data=train_path, vocab_size=100000, model=model_path)

In [ ]:
bpe = yttm.BPE(model=model_path)

In [ ]:
X_train[0]

'so what idiot stop vanalising'

In [ ]:
print(bpe.encode(X_train[0], output_type=yttm.OutputType.SUBWORD))

['▁so', '▁what', '▁idiot', '▁stop', '▁van', 'al', 'ising']


In [ ]:
vocab_len = len(bpe.vocab())
vocab_len

100000

In [ ]:
X_train_ind = [bpe.encode(text, output_type=yttm.OutputType.ID) for text in X_train]
X_test_ind = [bpe.encode(text, output_type=yttm.OutputType.ID) for text in X_test]

# Нейронная модель 

Ряд 4, колонка 5: Любая нейронная модель (минимум 5 слоев) с Dropout, Pooling и колбеками - 2 балла

In [ ]:
MAX_LEN = max(len(x) for x in X_train_ind)
MEAN_LEN = np.median([len(x) for x in X_train_ind])
MAX_LEN, MEAN_LEN

(3185, 40.0)

In [ ]:
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train_ind, maxlen=100)
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test_ind, maxlen=100)

In [ ]:
from tensorflow.keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
inputs = tf.keras.layers.Input(shape=(100,), name='inputs')
embeddings = tf.keras.layers.Embedding(input_dim=vocab_len, output_dim=10, name='embeddings')(inputs, )


pad1 = tf.keras.layers.Lambda(lambda x: tf.pad(x, 
                                               [[0,0], [77,77], [0, 0]], 
                                               mode='REFLECT'), name='pad1')(embeddings)
conv1 = tf.keras.layers.Conv1D(kernel_size=8, filters=56, strides = 2, name='conv1')(pad1)
drop1 = tf.keras.layers.Dropout(0.6, name='drop1')(conv1)
conv2 = tf.keras.layers.Conv1D(kernel_size=6, filters=40, strides = 2, name='conv2')(drop1)
pool1 = tf.keras.layers.MaxPool1D()(conv2)


drop0 = tf.keras.layers.Dropout(0.5, name='drop0')(embeddings)
pad2 = tf.keras.layers.Lambda(lambda x: tf.pad(x, 
                                               [[0,0], [85,85], [0, 0]], 
                                               mode='REFLECT'), name='pad2')(drop0)
conv3 = tf.keras.layers.Conv1D(kernel_size=10, filters=64, strides = 2, name='conv3')(pad2)
pool2 = tf.keras.layers.MaxPool1D()(conv3)
conv4 = tf.keras.layers.Conv1D(kernel_size=6, filters=32, strides = 2, name='conv4')(pool2)


concat = tf.keras.layers.concatenate([pool1, conv4], name='concat')
flat = tf.keras.layers.Flatten()(concat)
dense = tf.keras.layers.Dense(64, activation='relu')(flat)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=[f1, tf.keras.metrics.AUC()])

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('/content/drive/My Drive/Jigsaw/model.weights', 
                                                monitor='val_auc_12', 
                                                verbose=1, 
                                                save_weights_only=True, 
                                                save_best_only=True, 
                                                mode='max', 
                                                save_freq='epoch')
 
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_auc_12', 
                                              min_delta=0.01, 
                                              patience=3, 
                                              verbose=1, 
                                              mode='max')

In [ ]:
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inputs (InputLayer)             [(None, 100)]        0                                            
__________________________________________________________________________________________________
embeddings (Embedding)          (None, 100, 10)      1000000     inputs[0][0]                     
__________________________________________________________________________________________________
pad1 (Lambda)                   (None, 254, 10)      0           embeddings[0][0]                 
__________________________________________________________________________________________________
drop0 (Dropout)                 (None, 100, 10)      0           embeddings[0][0]                 
___________________________________________________________________________________________

In [ ]:
model.fit(X_train_pad, y_train, 
          validation_data=(X_test_pad, y_test),
          batch_size=1000,
          epochs=10,
          callbacks=[checkpoint, early_stop])

Epoch 1/10
202/202 [==============================] - ETA: 0s - loss: 0.2308 - f1: 0.3429 - auc_12: 0.8458
Epoch 00001: val_auc_12 improved from -inf to 0.94542, saving model to /content/drive/My Drive/Jigsaw/model.weights
202/202 [==============================] - 148s 732ms/step - loss: 0.2308 - f1: 0.3429 - auc_12: 0.8458 - val_loss: 0.1539 - val_f1: 0.6814 - val_auc_12: 0.9454
Epoch 2/10
202/202 [==============================] - ETA: 0s - loss: 0.1075 - f1: 0.7692 - auc_12: 0.9735
Epoch 00002: val_auc_12 improved from 0.94542 to 0.94855, saving model to /content/drive/My Drive/Jigsaw/model.weights
202/202 [==============================] - 148s 732ms/step - loss: 0.1075 - f1: 0.7692 - auc_12: 0.9735 - val_loss: 0.1501 - val_f1: 0.7151 - val_auc_12: 0.9485
Epoch 3/10
202/202 [==============================] - ETA: 0s - loss: 0.0622 - f1: 0.8768 - auc_12: 0.9914
Epoch 00003: val_auc_12 did not improve from 0.94855
202/202 [==============================] - 150s 742ms/step - loss: 0.

# Поиск аутлаеров

Ряд 7, колонка 1: Поиск аутлаеров в данных с помощью кластеризации (нужно найти хотя бы 20 странных не одинаковых текста)  - 1.5 балла

In [ ]:
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.cluster import DBSCAN

In [ ]:
X = data.comment_text.values

In [ ]:
tfidf = TfidfVectorizer(max_features=10000, min_df=5, ngram_range=(1,2), max_df=0.4)
X_tfidf = tfidf.fit_transform(X)

Получилось, что что-то более менее похожее на аутлаеры иногда появлялось в -1 кластере тогда, когда использовалось понижение размерности (я использовала PCA и TruncatedSVD)

In [ ]:
pca = PCA(n_components=2)
sample = X_tfidf[:10000]
X_pca = pca.fit_transform(sample.toarray())

In [ ]:
cluster = DBSCAN(min_samples = 5, eps = 0.02)
cluster.fit(X_pca)

DBSCAN(algorithm='auto', eps=0.02, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=5, n_jobs=None, p=None)

In [ ]:
set(cluster.labels_)

{-1, 0, 1, 2, 3, 4, 5, 6, 7, 8}

In [ ]:
out_inds = [i for i, label in enumerate(cluster.labels_) if label ==-1]

In [ ]:
out_inds[:10]

[539, 553, 584, 675, 821, 864, 963, 1437, 1611, 1908]

По большей части все равно собираются обычные комментарии, возможно с редкими словами

In [ ]:
data.loc[out_inds[:10], ['comment_text', 'toxic']]

,comment_text,toxic
539,"""\n\nElvis... Crying in the Chapel Feudalist """,0
553,iPad talk page \n\nWhy did you remove that IP's edit to the talk page? It wasn't spam...,0
584,"""\n\n Question on Possible Citation \n\nI'm sorry if I missed your question - I don't pay much a...",0
675,"""\n\n Please do not add unsourced negative or controversial biographical material to pages, as y...",0
821,"31 July 2007\n Please stop. If you continue to vandalize Wikipedia, you will be blocked from edi...",0
864,"""\n\nReplacement\n\nPLEASE DO NOT EDIT BELOW THIS NOTE. THE WHOLE SUPRA-SECTION WILL DISAPPEAR A...",0
963,"""\n\n Please do not remove content from pages without explanation, as you did with this edit to ...",0
1437,"""\nTalkback\n\nBlessed be!! Talk """,0
1611,"""\n\n Please stop your disruptive editing. If you continue to vandalize Wikipedia, you will be ...",0
1908,"""\n\nA tag has been placed on Beezwax, requesting that it be speedily deleted from Wikipedia. Th...",0


Но среди них можно найти "аутлаеры"

In [ ]:
print(data.loc[140864,'comment_text'])

"

hunk

hunk

YOU'VE BEEN HIT BY THE

|^^^^^^^^^^^^|
|SEXY TRUCK | '|"""""";.., ___.
|_..._...______===|= _|__|..., ] |
""(@ )'(@ )""""""""*|(@ )(@ )*****(@

ONCE YOU'VE BEEN HIT, YOU HAVE TO HIT 8 PEOPLE! IF YOU GET HIT AGAIN YOU'LL KNOW YOU'RE REALLY SEXY! IF YOU BRAKE THE CHAIN, YOULL BE CURSED WITH UN*SEXYNESS FOR 10 YEARS SO PASS IT; HIT WHO EVER YOU THINK IS SEXY!"


In [ ]:
numbers = [8468, 10137, 12833, 29023, 37571, 37861, 40220, 54528, 59587, 76063, 98371, 98599, 
           99858, 109927, 122666, 125021, 128827, 131729, 140545, 142181, 144580, 146571, 156684, 159601,
          7307, 38651, 1859, 21366, 1189, 201326, 21283, 22959, 47881, 7063]
numbers.sort()
print(len(numbers))

34


Ниже некоторые комментарии, найденные при разных параметрах кластеризации, которые содержат:

- слова на других языках

- странные ники пользователей

- бессмысленные сочетания букв

- слова написанные отдельными буквами (через пробел или через перенос строки)

- необычные символы (вроде ♥ ☺ ★ ♪ ߷ ♀)

- много небуквенных символов

In [ ]:
outliers = data.loc[numbers, ['comment_text', 'toxic']]
outliers

,comment_text,toxic
1189,"""\n{| style=""""background-color:#F5FFFA; padding:0;"""" cellpadding=""""0""""\n|style=""""border:1px soli...",0
1859,(T | C) \n\n Owned \n\n (T | C),0
7063,"You're a moron! \n\nIt's F\nU\nC\nK\n \nF\nU\nC\nK\n \nF\nU\nC\nK\n \nA\n \nD\nU\nC\nK\n,\n \nS\...",1
7307,I have placed the user who you said was the Sockpuppeters block log on the page. From what I hav...,0
8468,"""\nMuch content was dubiously changed and made what appeared to be changes to fundamental conten...",0
10137,"""==Name==\nI've been studying Satmar for a good bit, and I've been to Kiryas Joel. I've never he...",0
12833,Hi Raul!\nHi Raul!\nHi Raul!\nHi Raul!\nHi Raul!\nHi Raul!\nHi Raul!\nHi Raul!\nHi Raul!\nHi Rau...,0
21283,"""\nYeah that's not very many. I see """"searchFocus"""", """"Twinkle"""", """"HideFundraisingNotice"""", """"R...",0
21366,I SAID GOOD DAY BIATCH\nDONT BREAK WP:3RR BIATCH\nHAHAHAHAHAHAHAHAHAHHAAHAHHAHAHAAHHAHAHAHAHAHAH...,1
22959,95.svg|20px]] New Jersey Turnpike/Interstate 95\n|-\n!colspan=5|\nNew York\n|-\n|\n|\n|\n|rowspa...,0
